In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import data_reader as dr
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
from tensorflow import keras

In [3]:
data = dr.read()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2321 entries, 22 to 2483
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Date                 2321 non-null   object 
 1   Daily_trading_range  2321 non-null   float64
 2   Log_Volume_change    2321 non-null   float64
 3   Daily_return         2321 non-null   float64
 4   Daily_log_return     2321 non-null   float64
 5   Past_vol22           2321 non-null   float64
 6   gold                 2321 non-null   float64
 7   oil                  2321 non-null   float64
 8   Target22             2300 non-null   float64
 9   Target10             2312 non-null   float64
dtypes: float64(9), object(1)
memory usage: 199.5+ KB


In [4]:
data = data.drop(['Daily_return','Past_vol22'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2321 entries, 22 to 2483
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Date                 2321 non-null   object 
 1   Daily_trading_range  2321 non-null   float64
 2   Log_Volume_change    2321 non-null   float64
 3   Daily_log_return     2321 non-null   float64
 4   gold                 2321 non-null   float64
 5   oil                  2321 non-null   float64
 6   Target22             2300 non-null   float64
 7   Target10             2312 non-null   float64
dtypes: float64(7), object(1)
memory usage: 163.2+ KB


In [5]:
x_values = data.iloc[22:, 1:-2]
y_values = data[['Target22']]
y_values = y_values.iloc[22:]

In [6]:
x_values.info()
y_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2299 entries, 44 to 2483
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Daily_trading_range  2299 non-null   float64
 1   Log_Volume_change    2299 non-null   float64
 2   Daily_log_return     2299 non-null   float64
 3   gold                 2299 non-null   float64
 4   oil                  2299 non-null   float64
dtypes: float64(5)
memory usage: 107.8 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2299 entries, 44 to 2483
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Target22  2299 non-null   float64
dtypes: float64(1)
memory usage: 35.9 KB


In [7]:
scaler = MinMaxScaler()
scaled_x = scaler.fit_transform(x_values)
scaled_y = scaler.fit_transform(y_values)

In [8]:
split = int(len(x_values)*0.8)

In [10]:
window = 22
trainX = np.array(scaled_x[:split])
testX = np.array(scaled_x[split:])
trainY = np.array(scaled_y[:split])
testY = np.array(scaled_y[split:])

trainX.shape

(1839, 5)

In [11]:
Xtrain = []
ytrain = []
Xtest = []
ytest = []

for i in range(window, len(trainX)):
    Xtrain.append(trainX[i-window:i, :trainX.shape[1]])
    ytrain.append(trainY[i])
for i in range(window, len(testX)):
    Xtest.append(testX[i-window:i, :testX.shape[1]])
    ytest.append(testY[i])

In [12]:
Xtrain, ytrain = (np.array(Xtrain), np.array(ytrain))
Xtrain = np.reshape(Xtrain, (Xtrain.shape[0], Xtrain.shape[1], Xtrain.shape[2]))

Xtest, ytest = (np.array(Xtest), np.array(ytest))
Xtest = np.reshape(Xtest, (Xtest.shape[0], Xtest.shape[1], Xtest.shape[2]))

In [13]:
print(Xtrain.shape)
print(ytrain.shape)
print("-----")
print(Xtest.shape)
print(ytest.shape)

(1817, 22, 5)
(1817, 1)
-----
(438, 22, 5)
(438, 1)


In [15]:
model = Sequential()
model.add(Dense(25, input_shape=(Xtrain.shape[1],Xtrain.shape[2]), activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(Xtrain, ytrain, epochs=150, batch_size=16, validation_split=0.1, shuffle=False)

Epoch 1/150
103/103 [==============================] - 1s 7ms/step - loss: 0.0243 - val_loss: 0.0055
Epoch 2/150
103/103 [==============================] - 0s 2ms/step - loss: 0.0240 - val_loss: 0.0064
Epoch 3/150
103/103 [==============================] - 0s 2ms/step - loss: 0.0242 - val_loss: 0.0065
Epoch 4/150
103/103 [==============================] - 0s 2ms/step - loss: 0.0245 - val_loss: 0.0070

In [16]:
forecast = model.predict(Xtest)

In [17]:
rev_forecast = scaler.inverse_transform(forecast)
rev_ytest = scaler.inverse_transform(ytest)

ValueError: Found array with dim 3. Estimator expected <= 2.